<a href="https://colab.research.google.com/github/buketugurlu/dentaLyze/blob/main/yolov11_deneme4v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 896.9/896.9 kB 19.0 MB/s eta 0:00:00


In [4]:
pip install opencv-python-headless


In [5]:
pip install pyyaml


In [6]:
pip install albumentations


In [10]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [13]:
import os
from ultralytics import YOLO
import yaml

# Modeli eğitme
def train_model(dataset_yaml_path):
    model = YOLO('/content/drive/MyDrive/models/yolo11x.pt')
    model.train(
        data=dataset_yaml_path,  # YAML dosyasındaki veri seti bilgileri
        epochs=100,             # Eğitim epoch sayısı
        batch=16,               # Batch boyutu
        imgsz=640,              # Görüntü boyutu
        name='dental_detection' # Eğitim sonucu kaydedilecek dizin adı
    )

# Tahmin işlemi
def predict_images(model_path, test_images_dir):
    model = YOLO(model_path)  # Eğitimden en iyi ağırlıkları yükle
    results = model.predict(
        source=test_images_dir,  # Test görüntülerinin bulunduğu dizin
        conf=0.25,               # Güven eşiği
        save=True,               # Tahmin edilen görüntüleri kaydet
        save_txt=True,           # Tahmin sonuçlarını txt dosyası olarak kaydet
        save_conf=True           # Tahmin sonuçlarına güven skoru ekle
    )
    return results

# Ana fonksiyon
def main():
    try:
        # Dosya yolları
        augmented_data_dir = "/content/drive/MyDrive/Dental OPG XRAY Dataset/Augmented_Data"
        train_dir = os.path.join(augmented_data_dir, "train")
        valid_dir = os.path.join(augmented_data_dir, "valid")
        test_dir = os.path.join(augmented_data_dir, "test/images")
        dataset_yaml_path = os.path.join(augmented_data_dir, "dataset.yaml")

        # YAML dosyasını oluştur
        print("YAML dosyası oluşturuluyor...")
        yaml_content = {
            "path": augmented_data_dir,
            "train": "train/images",  # Eğitim görüntülerinin yolu
            "val": "valid/images",   # Doğrulama görüntülerinin yolu
            "test": "test/images",   # Test görüntülerinin yolu (opsiyonel)
            "nc": 6,                 # Sınıf sayısı
            "names": ['Healthy Teeth', 'Caries', 'Impacted Teeth', 'BDC/BDR', 'Infection', 'Fractured Teeth']
        }
        with open(dataset_yaml_path, "w") as yaml_file:
            yaml.dump(yaml_content, yaml_file)

        # Modeli eğit
        print("Eğitim başlatılıyor...")
        train_model(dataset_yaml_path)

        # Test görüntüleri üzerinde tahmin yap
        print("Tahmin işlemi başlatılıyor...")
        results = predict_images('runs/train/dental_detection/weights/best.pt', test_dir)
        print("Tahmin tamamlandı.")
        print(results)

    except Exception as e:
        print(f"Hata oluştu: {e}")

if __name__ == "__main__":
    main()


YAML dosyası oluşturuluyor...
Eğitim başlatılıyor...
Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/models/yolo11x.pt, data=/content/drive/MyDrive/Dental OPG XRAY Dataset/Augmented_Data/dataset.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=dental_detection, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, r

100%|██████████| 755k/755k [00:00<00:00, 14.5MB/s]


Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1      2784  ultralytics.nn.modules.conv.Conv             [3, 96, 3, 2]                 
  1                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  2                  -1  2    389760  ultralytics.nn.modules.block.C3k2            [192, 384, 2, True, 0.25]     
  3                  -1  1   1327872  ultralytics.nn.modules.conv.Conv             [384, 384, 3, 2]              
  4                  -1  2   1553664  ultralytics.nn.modules.block.C3k2            [384, 768, 2, True, 0.25]     
  5                  -1  1   5309952  ultralytics.nn.modules.conv.Conv             [768, 768, 3, 2]              
  6                  -1  2   5022720  ultralytics.nn.modules.block.C3k2            [768, 768, 2, True]           
  7                  -1  1   5309952  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 67.7MB/s]


AMP: checks passed ✅


...

    100/100      16.5G     0.8258     0.7626       1.07          3        640: 100%|██████████| 94/94 [00:23<00:00,  4.02it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:01<00:00,  6.48it/s]

                   all        383        546        0.1      0.504     0.0841     0.0539



100 epochs completed in 0.774 hours.
Optimizer stripped from runs/detect/dental_detection/weights/last.pt, 114.4MB
Optimizer stripped from runs/detect/dental_detection/weights/best.pt, 114.4MB

Validating runs/detect/dental_detection/weights/best.pt...
Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
YOLO11x summary (fused): 464 layers, 56,833,954 parameters, 0 gradients, 194.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:02<00:00,  4.95it/s]


                   all        383        546        0.1      0.508     0.0839     0.0539
         Healthy Teeth         53         91      0.103      0.527     0.0674     0.0441
                Caries         14         23      0.139      0.478      0.108     0.0654
        Impacted Teeth         37         86      0.123      0.547      0.092     0.0554
               BDC/BDR          7          7      0.128      0.714      0.137      0.102
             Infection         21         36     0.0518      0.444     0.0585     0.0299
       Fractured Teeth        100        303     0.0588       0.34     0.0408     0.0265
Speed: 0.1ms preprocess, 2.3ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/dental_detection
Tahmin işlemi başlatılıyor...
Hata oluştu: [Errno 2] No such file or directory: 'runs/train/dental_detection/weights/best.pt'
